<img src="https://cdn.comet.ml/img/notebook_logo.png">

<img align="right" src="https://docs.nerf.studio/_images/logo.png" width="400">


This notebook shows to use [Comet](https://www.comet.com/site/?utm_source=nerf&utm_medium=referral&utm_content=colab) with Nerfstudio: a popular library that provides a simple API for a simplified end-to-end process of creating, training, and testing NeRFs.

# Install dependencies

In [ ]:
# Install pytorch with Cuda

%cd /content/
%pip install --upgrade pip
%pip install -U torch==2.0.1+cu118 torchvision==0.15.2+cu118 torchaudio==2.0.2+cu118 --extra-index-url https://download.pytorch.org/whl/cu118

In [ ]:
# Installing TinyCuda
%cd /content/
!gdown "https://drive.google.com/u/1/uc?id=1-7x7qQfB7bIw2zV4Lr6-yhvMpjXC84Q5&confirm=t"
%pip install tinycudann-1.7-cp310-cp310-linux_x86_64.whl

In [ ]:
# Install nerfstudio
%cd /content/
%pip install "nerfstudio>=1.0.0"

In [ ]:
# Install dependency to run the training with logs (see https://github.com/nerfstudio-project/nerfstudio/pull/2870 for more context)
%pip install colab-xterm

# Download the training dataset

In [ ]:
%%time
%cd /content/
!ns-download-data sitcoms3d

# Setup everything

Create your free account at [Comet.com](https://www.comet.com/site/?utm_source=nerf&utm_medium=referral&utm_content=colab) and grab your API key which can be found under account settings.

In [ ]:
# Setup Comet

import comet_ml

comet_ml.init(project_name="comet-example-nerfstudio-example")

# Training

In [ ]:
# First let's import everything that we need and define some helper functions
import os
import uuid
from pathlib import Path


def generate_guid():
    # type: () -> str
    """Generate a GUID"""
    return uuid.uuid4().hex

In [ ]:
# Let's define some parameters
MAX_ITERATIONS = 30000
NERF_EXPERIMENT_NAME = "TBBT-nerfstudio"

In [ ]:
# Let's generate an unique Comet Experiment Key, this will allows us to save everything to a single experiment
exp_key = generate_guid()
os.environ["COMET_EXPERIMENT_KEY"] = exp_key

In [ ]:
# Start the training in a real terminal to see logs
%load_ext colabxterm
%env TERM=xterm

from IPython.display import clear_output

clear_output(wait=True)

print(
    "\033[1m"
    + "Copy and paste the following command into the terminal window that pops up under this cell."
    + "\033[0m"
)
print(
    f"ns-train nerfacto --vis viewer+comet --viewer.websocket-port 7007 --viewer.make-share-url True --project-name comet-example-TBBT-nerfstudio --experiment-name {NERF_EXPERIMENT_NAME} --max-num-iterations {MAX_ITERATIONS} --logging.steps-per-log 100 sitcoms3d-data --data data/sitcoms3d/TBBT-big_living_room/ --downscale-factor 4"
)
print()

%xterm

In [ ]:
# Train the model
%cd /content
!env PYTHONUNBUFFERED=true ns-train nerfacto \
    --vis comet \
    --logging.local-writer.max-log-size 0 \
    --project-name comet-example-TBBT-nerfstudio \
    --experiment-name {NERF_EXPERIMENT_NAME} \
    --max-num-iterations {MAX_ITERATIONS}\
    --pipeline.datamanager.images-on-gpu True\
    --logging.steps-per-log 100\
    sitcoms3d-data \
    --data data/sitcoms3d/TBBT-big_living_room/ \
    --downscale-factor 4

In [ ]:
# Log the model to Comet
base_dir = Path("./outputs/") / NERF_EXPERIMENT_NAME / "nerfacto"
# Get the latest trained model
training_run_dir = sorted(base_dir.iterdir())[-1]
print(training_run_dir)

In [ ]:
experiment = comet_ml.ExistingExperiment(experiment_key=exp_key)
experiment.log_model(
    "nerfstudio-model",
    str(training_run_dir),
    prepend_folder_name=False,
    metadata={"nerfstudio_project_dir": str(training_run_dir)},
)

In [ ]:
# Generate validation videos
# Get the validation camera paths from Comet
config_filename = training_run_dir / "config.yml"

artifact = experiment.get_artifact("TBTT-Nerfstudio-Camera-Paths", workspace="examples")
output_path = Path("./validation/")
artifact.download(output_path)

# And for each of them, generate a video that we log to Comet
for camera_path in (output_path / "camera_paths").glob("camera_path_*.json"):
    validation_name = camera_path.stem
    output_video_path = "./renders/%s.mp4" % (validation_name)

    !ns-render camera-path --load-config $config_filename --camera-path-filename $camera_path --output-path $output_video_path

    experiment.log_video(output_video_path)

In [ ]:
# And finally closes the Comet Experiment to save the Notebook code and flush everything to Comet
experiment.end()